<img src="https://www.luxonis.com/logo.svg" width="400">

# Training an Instance Segmentation Model from a Roboflow Dataset

## 🌟 Overview
In this tutorial, we'll go through the process of training a custom instance segmentation model using a dataset from Roboflow Universe. We'll find a suitable dataset and import it to LuxonisTrain pipeline. We'll also validate the performance of our model, export it, and make it ready for deployment on a Luxonis device.

## 📜 Table of Contents
- [🛠️ Installation](#️installation)
- [🗃️ Data Preparation](#data-preparation)
- [🏋️‍♂️ Training](#️️training)
    - [⚙️ Configuration](#️configuration)
    - [🦾 Train](#train)
- [✍ Test](#test)
    - [🧠 Infer](#infer)
- [🗂️ Export and Archive](#export-and-archive)
- [🤖 Deploy](#deploy)
- [📷 DepthAI Script](#depthai-script)

<a name="️installation"></a>

## 🛠️ Installation


The main focus of this tutorial is using [`LuxonisTrain`](https://github.com/luxonis/luxonis-train), a user-friendly tool designed to streamline the training of deep learning models, especially for edge devices. We'll also use [`LuxonisML`](https://github.com/luxonis/luxonis-ml) since it provides us with a collection of utility functionality. We will download the dataset from a popular open-source annotated data repostiory - [Roboflow](https://roboflow.com/).

In [ ]:
%pip install -q datasets>=3.1.0 luxonis-train==0.3.8 -U

<a name ="data-preparation"></a>

## Data Preparation

Find a dataset that best matches your use case on [Roboflow Universe](https://universe.roboflow.com/). For this demo we chose [PCB defects dataset](https://universe.roboflow.com/learn-uzoux/pcb-defect-0i1a7) that has annotated soldering quality for SMD components.

To get the dataset in for training go to `Dataset tab` in the Roboflow project page, select the desired version and copy the URL. From this URL you will need to extract the data in order for the final line to look like: 

    roboflow://<TEAM_NAME>/<DATASET_NAME>/<DATASET_VERSION>/coco

Our URL was: https://universe.roboflow.com/learn-uzoux/pcb-defect-0i1a7 and the end line looks like:

    roboflow://learn-uzoux/pcb-defect-0i1a7/2/coco

You will need this in the following step. 

💡 Tip (Optional): If the original dataset is not good enough for your usecase, you can expand it by uploading your own custom images and annotations using the Roboflow web interface. Follow Roboflow's [official tutorial](https://blog.roboflow.com/getting-started-with-roboflow/) here to get started.

<a name ="️️training"></a>

## 🏋️‍♂️ Training

<a name ="️configuration"></a>

### ⚙️ Configuration

We have prepared the dataset and are almost ready for the actual training. The last step is just to set up our training configuration file. The whole training process in `LuxonisTrain` doesn't require any coding. We advise you to take one of the base configuration files from [here](https://github.com/luxonis/luxonis-train/tree/main/configs) depending on the task, and then edit it to fit your needs.

In our case, since we are training an instance segmentation model, we'll take a [`instance_segmentation_light_model.yaml`](https://github.com/luxonis/luxonis-train/blob/main/configs/instance_segmentation_light_model.yaml) config as a starting point, which downloads pre-trained COCO weights, making it ideal for fine-tuning. There are many parameters that we can change, and we advise you to go through the [`documentation`](https://github.com/luxonis/luxonis-train/blob/main/configs/README.md) to find all of them. In this tutorial, we'll only go through some basic ones to get you started on your journey.

Below is a starting point for our config. As mentioned before, we already made some changes to it, so it works with this tutorial (model name and dataset name change), but feel free to edit it further and make it your own. When you are done editing, you can execute the cell, and the file will be written and ready to use.

Make sure you point the training of the model to your own dataset by changing the parameter `dataset_dir`.

**Note**: In case you don't have enough computing power on your machine, you can either use [Google Colab](https://colab.research.google.com/) (with GPU enabled), or you can try tweaking the training hyperparameters (such as lowering number of epochs or batch size). However, please be aware that bad parametrization can result in worse performance.

In [ ]:
%%writefile solder_defect_segmentation_model_config.yaml
model:
  name: pcb_defects_instance_seg
  predefined_model:
    name: InstanceSegmentationModel
    params:
      variant: heavy
      loss_params:
        bbox_loss_weight: 60 # Should be 7.5 * accumulate_grad_batches for best results
        class_loss_weight: 4 # Should be 0.5 * accumulate_grad_batches for best results
        dfl_loss_weight: 12 # Should be 1.5 * accumulate_grad_batches for best results

loader:
  params:
    dataset_dir: "roboflow://learn-uzoux/pcb-defect-0i1a7/2/coco" # Change this link to your dataset

rich_logging: false

trainer:
  precision: "16-mixed"
  preprocessing:
    train_image_size: [384, 512]
    keep_aspect_ratio: true
    normalize:
      active: true
      params:
        mean: [0., 0., 0.]
        std: [1, 1, 1]
    augmentations:
    - name: Rotate
      params:
        limit: 3
        p: 0.5
        border_mode: 0
        value: [0, 0, 0]
    - name: Perspective
      params:
        scale: [0.02, 0.04]
        keep_size: true
        pad_mode: 0
        pad_val: 0
        mask_pad_val: 0
        fit_output: false
        interpolation: 1
        always_apply: false
        p: 0.5
    - name: Affine
      params:
        scale: 1
        translate_percent: 0
        translate_px: null
        shear: 5
        interpolation: 1
        mask_interpolation: 0
        cval: 0
        cval_mask: 0
        mode: 0
        fit_output: false
        keep_ratio: false
        rotate_method: largest_box
        always_apply: false
        p: 0.4
    - name: MotionBlur
      params:
        blur_limit: [2,4]
        p: 0.1
    - name: ToGray
      params:
        p: 0.5
    - name: RandomBrightnessContrast
      params:
        ensure_safe_range: True
        p: 0.3
    - name: GaussNoise
      params:
        std_range: [0.0, 0.05]
        p: 0.1

  batch_size: 8
  epochs: &epochs 200
  n_workers: 4
  validation_interval: 10
  n_log_images: 8
  gradient_clip_val: 10

  callbacks:
    - name: EMACallback
      params:
        decay: 0.9999
        use_dynamic_decay: True
        decay_tau: 2000
    - name: ExportOnTrainEnd
    - name: TestOnTrainEnd
    - name: GradientAccumulationScheduler
      params:
        scheduling: # warmup phase is 3 epochs
          0: 1
          1: 4
          2: 8 # For best results, always accumulate gradients to effectively use 64 batch size

  training_strategy:
    name: "TripleLRSGDStrategy"
    params:
      warmup_epochs: 3
      warmup_bias_lr: 0.0
      warmup_momentum: 0.8
      lr: 0.01
      lre: 0.0001
      momentum: 0.937
      weight_decay: 0.0005
      nesterov: True
      cosine_annealing: False

<a name ="train"></a>

### 🦾 Train

To start the training, we need to initialize the `LuxonisModel`, pass it the path to the configuration file, and call the `train()` method on it. If you are training the model on your private Roboflow dataset you will have to enter also your Roboflow API KEY.

**Note**: LuxonisTrain also supports all these commands through usage of its CLI ([docs here](https://github.com/luxonis/luxonis-train/tree/main?tab=readme-ov-file#-cli)), no code required. We won't use them for tutorial purposes, but feel free to use them when you do it yourself.

In [ ]:
from luxonis_train import LuxonisModel
import os

# If you are using your private dataset from Roboflow, you must enter your API key
os.environ['ROBOFLOW_API_KEY'] = '<ADD_YOUR_ROBOFLOW_API_KEY_HERE>'
config_path = "solder_defect_segmentation_model_config.yaml"

luxonis_model = LuxonisModel(config_path)
luxonis_model.train()

`LuxonisTrain` has also already implemented automatic tracking of training runs. By default, `Tensorboard` is used, and to look at the losses, metrics, and visualizations during training, we can inspect the logs. If you check the `output` folder, you'll see that every run creates a new directory, and each run also has its training logs in the `./output/tensorboard_logs` where the name of the folder matches the run's name. To make all the subsequent commands work automatically, please set the name of your run below.

In [ ]:
RUN_NAME = "<YOUR_RUN_NAME>"

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output/tensorboard_logs # TODO: Change the name of the training run

In [ ]:
!tensorboard --logdir output/tensorboard_logs

<a name ="test"></a>

## ✍ Test

Now, we have a trained model that performs well on the validation set. The next step is to check its performance on the testing set, a collection of images we've kept hidden from the model. It should only be used to evaluate whether the model is good objectively. Since this is an instance segmentation task, we use the Mean Average Precision, Mean Average Recall, and F1 Score metrics to check the model performance quantitatively.

If you check out the run directory, you'll see two folders inside: `best_val_metric` and `min_val_loss`. Both have checkpoint files generated during training based on best validation metric performance and minimal validation loss. For evaluation, we'll want to use one of these checkpoints; we recommend that you use one that has the lowest validation loss.

In [ ]:
weights = luxonis_model.get_min_loss_checkpoint_path() # gets checkpoint where validation loss was the lowest
# weights = luxonis_model.get_best_metric_checkpoint_path() # gets checkpoint where validation metric was the highest

metrics = luxonis_model.test(view="test", weights=weights)

<a name ="infer"></a>

### 🧠 Infer

Usually, we also want to visualize the prediction of the trained model on test images to ensure it does what it is supposed to do. This is called inference, and we can perform it either on one of the views (e.g., test) or a random image, directory of images, or whole video (for more details, refer to the [docs](https://github.com/luxonis/luxonis-train/tree/main?tab=readme-ov-file#inference)). In our case, we'll infer the model on test images.

In [ ]:
# NOTE: If you are using Google Colab use this and images will be saved to "infer_results_beans_model" directory
luxonis_model.infer(weights=weights, save_dir="infer_results_defects_model2", view="test")

# NOTE: If you are not using Google Colab use this and images will be displayed
# luxonis_model.infer(
#     weights=weights,
#     view="test"
# ) 

Now, let's visualize one of the predictions.

In [ ]:
from IPython.display import Image
Image(filename='infer_results_defects_model2/-PrecisionSegmentBBoxHead_InstanceSegmentationVisualizer_3.png')

<a name ="export-and-archive"></a>

## 🗂️ Export and Archive

Once the model is trained and tested, we want to prepare it for deployment on the device. This preparation consists of 2 steps. First, we want to export the model trained with PyTorch to a more general format called [`Open Neural Network Exchange (ONNX)`](https://onnx.ai/). Then, we want to package this exported model with all the metadata containing information about the inputs, outputs, and training configuration used. This is called archiving. These steps can be done quickly with just one command in `LuxonisTrain`.

In [ ]:
archieve_path = luxonis_model.archive(weights=weights)
print("Model archieved to:", archieve_path)

Notice that two new folders were created in our run directory. One is called `export` and has an ONNX model while the other is called `archive` which has `.tar.xz` file. The tar file is a compressed file that holds the aforementioned ONNX model with all the model metadata.

To use the NN archive for conversion, please copy the path to the .tar.xz file:

In [ ]:
NN_ARCHIVE_PATH = "output/6-emerald-wildfowl/archive/pcb_defects_instance_seg.onnx.tar.xz"

<a name ="deploy"></a>

## 🤖 Deploy

Now that we have successfully exported and archived the model, we aim to deploy it to the Luxonis device. The model's specific format depends on the Luxonis device series you have. We will show you how to use our [`ModelConverter`](https://github.com/luxonis/modelconverter) to convert the model as simply as possible. 

We'll start by installing the `ModelConverter`.

In [ ]:
# %pip install -q modelconv==0.4.0 -U
%pip  install -U --no-deps "modelconv @ git+https://github.com/luxonis/modelconverter.git@fix/keyring-timeout"

We will use the `ModelConverter` Python API, which leverages our [`HubAI`](https://hub.luxonis.com) platform to perform model conversion in the background. To get started, you'll need to create an account on `HubAI` and obtain your team’s API key.

In [ ]:
HUBAI_API_KEY = "<YOUR_HUBAI_API_KEY>"

Model conversion can be done via either the CLI or the Python API — here, we'll use the latter. For more information, see the [online usage section](https://github.com/luxonis/modelconverter?tab=readme-ov-file#online-usage) of the documentation.

The call below creates a new model card within your team on `HubAI`, uploads the model file and metadata, then performs cloud-side conversion to the selected target platform (e.g., [`RVC2`](https://rvc4.docs.luxonis.com/hardware/platform/rvc/rvc2/), [`RVC4`](https://rvc4.docs.luxonis.com/hardware/platform/rvc/rvc4/)). Once completed, the converted model is automatically downloaded to your device.

For HubAI-specific conversion parameters, refer to the [online conversion section](https://github.com/luxonis/modelconverter/tree/e6a3478ba47d8f92d4d60217f2aee0f4f468cb14/modelconverter/hub#online-conversion) of the ModelConverter documentation. Platform-specific parameters are also documented there.


In [ ]:
from modelconverter import convert

converted_model = convert.RVC4(
    api_key=HUBAI_API_KEY,
    path=NN_ARCHIVE_PATH,
    name="Solder Defect Instance Segmentation1",
    description_short="Solder Defect Instance Segmentation Model",
    tasks=["INSTANCE_SEGMENTATION"],
    license_type="MIT",
    is_public=False
)

We have successfully converted our trained model for an RVC4 device, so let's test it! Please copy the path to the downloaded archive with the converted model from the output log of the last code cell; we will use it in the next section.

In [ ]:
MODEL_PATH = "<YOUR_DOWNLOADED_MODEL_ARCHIVE_PATH>"

<a name ="depthai-script"></a>

## 📷 DepthAI Script

To test our model on one of our cameras, we first need to install [`DepthAI`](https://rvc4.docs.luxonis.com/software/) in version 3 and [`DepthAI Nodes`](https://rvc4.docs.luxonis.com/software/ai-inference/depthai-nodes/). Moreover, the script we'll write must run locally and require a Luxonis device connected to your machine.

In [ ]:
%pip install -q depthai==3.0.0rc2 -U
%pip install -q depthai-nodes==0.3.0 -U

To run the model on a DepthAI device using the script below, please note the following:

- You can view the output stream by opening [http://localhost:8082](http://localhost:8082) in your browser.

- If you're running the script from a Jupyter Notebook, the output may not appear directly within the notebook. The script should print a link pointing to [http://localhost:8082](http://localhost:8082) for accessing the stream.

- To stop the video stream, press **`q`** while focused on the visualizer page.

In [ ]:
DEVICE = None # Set to None to use the default device, or you can specify a specific device IP

In [ ]:
import depthai as dai
from depthai_nodes.node import ParsingNeuralNetwork, ApplyColormap, ImgFrameOverlay

device = dai.Device(dai.DeviceInfo(DEVICE)) if DEVICE else dai.Device()
platform = device.getPlatform()
img_frame_type = dai.ImgFrame.Type.BGR888i if platform.name == "RVC4" else dai.ImgFrame.Type.BGR888p
visualizer = dai.RemoteConnection(httpPort=8082)

with dai.Pipeline(device) as pipeline:
    cam = pipeline.create(dai.node.Camera).build()
    nn_archive = dai.NNArchive(MODEL_PATH)
    # Create the neural network node
    nn_with_parser = pipeline.create(ParsingNeuralNetwork).build(
        cam.requestOutput((640, 640), type=img_frame_type, fps=30), 
        nn_archive
    )
    # transform output array to colormap
    apply_colormap_node = pipeline.create(ApplyColormap).build(nn_with_parser.out)
    # overlay frames
    overlay_frames_node = pipeline.create(ImgFrameOverlay).build(
        nn_with_parser.passthrough,
        apply_colormap_node.out,
    ) 
    # Configure the visualizer node
    visualizer.addTopic("Video", overlay_frames_node.out, "images")
    visualizer.addTopic("Detections", nn_with_parser.out, "detections")

    pipeline.start()
    visualizer.registerPipeline(pipeline)

    while pipeline.isRunning():
        key = visualizer.waitKey(1)
        if key == ord("q"):
            print("Got q key from the remote connection!")
            break

Yay! 🎉🎉🎉 Huge congratulations, you have successfully finished this tutorial in which you deployed an instance segmentation model trained using `luxonis-train` on a Cracks and Spallings dataset to our camera!